In [11]:
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('../')
import pandas as pd
from one.api import ONE, One
from functions.neurometric import fit_get_shift_range
from braindelphi.decoding.functions.process_targets import optimal_Bayesian
import models.utils as mut
from scipy.stats import pearsonr, spearmanr, wilcoxon
import brainbox.io.one as bbone

from braindelphi.decoding.settings import kwargs
from braindelphi.params import CACHE_PATH, IMPOSTER_SESSION_PATH
from braindelphi.decoding.functions.utils import load_metadata
import pickle

from tqdm import tqdm


In [9]:
# get all eids 
bwmdf, _ = load_metadata(CACHE_PATH.joinpath('*_%s_metadata.pkl' % kwargs['neural_dtype']).as_posix())
eids = bwmdf['dataset_filenames']['eid'].values
eids.shape

(259,)

In [12]:
outdict = {}
nb_simul_beh_shift = 10000

oracle_pLefts = []
priors_pLefts = []
#  session level
for eid in tqdm(eids):
    print(eid)
    try:
        data =  bbone.load_trials_df(eid)
    except:
        print('loading error')
        continue
    uniq_contrasts = np.array([-1., -0.25, -0.125, -0.0625, 0.,  0.0625,  0.125, 0.25,  1.])

    side, stim, act, oracle_pLeft = mut.format_data(data)
    prior = optimal_Bayesian(act, side).numpy()
    oracle_pLefts.append(oracle_pLeft)
    priors_pLefts.append(prior)

    #trials_df = bbone.load_trials_df(eid, one=one, addtl_types=['firstMovement_times'])
    #reaction_times = (trials_df['firstMovement_times'] - trials_df['goCue_times']).values

    # take out negative reaction times and when the mouse doesn't perform an action
    stim = stim[(act != 0)]
    side = side[(act != 0)]
    prior = prior[(act != 0)]
    act = act[(act != 0)]

    pLeft_constrast = {c: np.mean(act[stim == c] == 1) for c in uniq_contrasts}
    no_integration_act = np.vstack([2 * np.random.binomial(1, pLeft_constrast[c], size=nb_simul_beh_shift) - 1 for
                                    c in stim])

    perfat0 = (act == side)[stim == 0].mean()
    t, p = wilcoxon((act == side)[stim == 0] * 1 - 0.5, alternative='greater')

    p_nointegration = np.mean((no_integration_act == side[:, None]).mean(axis=0) < (act == side).mean())

    p_0cont_nointegration = np.mean((no_integration_act[stim == 0] == side[stim == 0, None]).mean(axis=0)
                                    < (act == side)[stim == 0].mean())

    low_prob_idx_trials = [(prior < 0.5) * (stim == c) for c in uniq_contrasts]
    lowprob_arr = [uniq_contrasts,
                   [len(act[idx]) for idx in low_prob_idx_trials],
                   [(act[idx] == 1).mean() for idx in low_prob_idx_trials]]
    high_prob_idx_trials = [(prior > 0.5) * (stim == c) for c in uniq_contrasts]
    highprob_arr = [uniq_contrasts,
                    [len(act[idx]) for idx in high_prob_idx_trials],
                    [(act[idx] == 1).mean() for idx in high_prob_idx_trials]]

    full_neurometric = fit_get_shift_range([lowprob_arr, highprob_arr], False, nfits=500)

    outdict[eid] = [t, p, p_nointegration, p_0cont_nointegration, perfat0, full_neurometric['shift']]

behdf = pd.DataFrame.from_dict(outdict, orient='index', columns=['tvalue', 'pvalue', 'p_nointegration',
                                                                 'p_0cont_nointegration',
                                                                 '0contrast_perf', 'psychometric_shift'])



  0%|          | 0/2 [00:00<?, ?it/s]

56956777-dca5-468c-87cb-78150432cc57


Inconsistent dimensions for object: trials 
(425,),	stimOff_times
(425,),	goCueTrigger_times
(421,),	itiDuration
(425, 2),	intervals_bpod
(425,),	response_times
(425,),	choice
(425,),	feedback_times
(425,),	feedbackType
(425,),	probabilityLeft
(425,),	goCue_times
(425,),	firstMovement_times
(425,),	contrastLeft
(425,),	contrastRight
(425,),	stimOn_times
(425,),	rewardVolume
(425, 2),	intervals
 50%|█████     | 1/2 [00:47<00:47, 47.71s/it]

a8a8af78-16de-4841-ab07-fde4b5281a03


Inconsistent dimensions for object: trials 
(550, 2),	intervals_bpod
(550,),	goCueTrigger_times
(550,),	stimOff_times
(549,),	itiDuration
(550,),	contrastLeft
(550,),	probabilityLeft
(550,),	feedbackType
(550,),	feedback_times
(550,),	response_times
(550,),	stimOn_times
(550,),	rewardVolume
(550,),	firstMovement_times
(550,),	choice
(550,),	contrastRight
(550,),	goCue_times
(550, 2),	intervals
100%|██████████| 2/2 [01:35<00:00, 47.93s/it]


In [13]:
behdf.to_parquet('beh_shift_session_10-08-2022.parquet')

,tvalue,pvalue,p_nointegration,p_0cont_nointegration,0contrast_perf,psychometric_shift
56956777-dca5-468c-87cb-78150432cc57,728.5,0.020124,0.8654,0.9879,0.673913,0.392902
a8a8af78-16de-4841-ab07-fde4b5281a03,1165.5,0.063754,0.8014,0.9459,0.596774,0.197099
